In [1]:
import re
import kss
f = open('4BH00005.txt', encoding="utf8")
i=0
sentence_list = []
while True:
    line = f.readline()
    if not line: break
    if line != '\n':
        i=i+1
        line = re.sub('<.+?>', '', line, 0, re.S)
        line = re.sub('\(.+?\)', '', line, 0, re.S)
        line = re.sub('\[.+?\]', '', line, 0, re.S)
        line = re.sub('\{.+?\}', '', line, 0, re.S)
        line = re.sub('《.+?》', '', line, 0, re.S)
        line = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!『』\‘|\(\)\[\]\<\>`\'…《》\{\}_「」±√]', '', line,0,re.S)
        line = line.lower()
        if len(line) >10:
            sentence_list+= kss.split_sentences(line)
f.close()
print(len(sentence_list))

2264


In [2]:
print(sentence_list[:5])

['제1장 531 교육개혁의 정당화논리 탐색', '이 장에서는 문민정부 시절 교육개혁위원회가 마련한 531 교육개혁안의 정당화논리를 분석하고 있다', '분석을 통해 개혁의 목표가 창의력 배양과 교육의 질적 수월성 추구에 있음을 밝히고 있다', '개혁주체들은 교육공급 측면에서 경쟁조건을 창출함으로써 교육서비스의 질을 높일 수 있다고 주장한다 한마디로 시장경제의 원리에 사로잡혀 있는 것이다', '교육체제를 운영하는 데는 두 가지 상반되는 가치가 존재한다 하나는 평등이고 다른 하나는 수월성이다']


In [53]:
choseong_list = [char for char in "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"]
word_list = []
for sentence in sentence_list:
    sen2word = list()
    for word in sentence:
        if re.match('[가-힣]',word) is not None :
            sen2word+=choseong_list[(int)((ord(word)-ord('가'))/(21*28))]
    word_list.append(sen2word)

In [58]:
print(word_list[:5])

[['ㅈ', 'ㅈ', 'ㄱ', 'ㅇ', 'ㄱ', 'ㅎ', 'ㅇ', 'ㅈ', 'ㄷ', 'ㅎ', 'ㄴ', 'ㄹ', 'ㅌ', 'ㅅ'], ['ㅇ', 'ㅈ', 'ㅇ', 'ㅅ', 'ㄴ', 'ㅁ', 'ㅁ', 'ㅈ', 'ㅂ', 'ㅅ', 'ㅈ', 'ㄱ', 'ㅇ', 'ㄱ', 'ㅎ', 'ㅇ', 'ㅇ', 'ㅎ', 'ㄱ', 'ㅁ', 'ㄹ', 'ㅎ', 'ㄱ', 'ㅇ', 'ㄱ', 'ㅎ', 'ㅇ', 'ㅇ', 'ㅈ', 'ㄷ', 'ㅎ', 'ㄴ', 'ㄹ', 'ㄹ', 'ㅂ', 'ㅅ', 'ㅎ', 'ㄱ', 'ㅇ', 'ㄷ'], ['ㅂ', 'ㅅ', 'ㅇ', 'ㅌ', 'ㅎ', 'ㄱ', 'ㅎ', 'ㅇ', 'ㅁ', 'ㅍ', 'ㄱ', 'ㅊ', 'ㅇ', 'ㄹ', 'ㅂ', 'ㅇ', 'ㄱ', 'ㄱ', 'ㅇ', 'ㅇ', 'ㅈ', 'ㅈ', 'ㅅ', 'ㅇ', 'ㅅ', 'ㅊ', 'ㄱ', 'ㅇ', 'ㅇ', 'ㅇ', 'ㅇ', 'ㅂ', 'ㅎ', 'ㄱ', 'ㅇ', 'ㄷ'], ['ㄱ', 'ㅎ', 'ㅈ', 'ㅊ', 'ㄷ', 'ㅇ', 'ㄱ', 'ㅇ', 'ㄱ', 'ㄱ', 'ㅊ', 'ㅁ', 'ㅇ', 'ㅅ', 'ㄱ', 'ㅈ', 'ㅈ', 'ㄱ', 'ㅇ', 'ㅊ', 'ㅊ', 'ㅎ', 'ㅇ', 'ㄹ', 'ㅆ', 'ㄱ', 'ㅇ', 'ㅅ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅇ', 'ㄴ', 'ㅇ', 'ㅅ', 'ㅇ', 'ㄷ', 'ㄱ', 'ㅈ', 'ㅈ', 'ㅎ', 'ㄷ', 'ㅎ', 'ㅁ', 'ㄷ', 'ㄹ', 'ㅅ', 'ㅈ', 'ㄱ', 'ㅈ', 'ㅇ', 'ㅇ', 'ㄹ', 'ㅇ', 'ㅅ', 'ㄹ', 'ㅈ', 'ㅎ', 'ㅇ', 'ㄴ', 'ㄱ', 'ㅇ', 'ㄷ'], ['ㄱ', 'ㅇ', 'ㅊ', 'ㅈ', 'ㄹ', 'ㅇ', 'ㅇ', 'ㅎ', 'ㄴ', 'ㄷ', 'ㄴ', 'ㄷ', 'ㄱ', 'ㅈ', 'ㅅ', 'ㅂ', 'ㄷ', 'ㄴ', 'ㄱ', 'ㅊ', 'ㄱ', 'ㅈ', 'ㅈ', 'ㅎ', 'ㄷ', 'ㅎ', 'ㄴ', 'ㄴ', 'ㅍ', 'ㄷ', 'ㅇ', 'ㄱ', 'ㄷ', 'ㄹ', 'ㅎ', 'ㄴ', 'ㄴ', 'ㅅ', 'ㅇ', 'ㅅ', 'ㅇ', 'ㄷ']]
